# Use age and sex to determine the presence of heart disease in patients of Hungary.

### Introduction



Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the ques

tion
Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your 


analysis.
Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize

 the results
Expected outcomes and significance:
What do you expect to find?
What impact could such findings have?
What future questions could this lead to?

### Introduction: 

Heart disease refers to a range of conditions that affect the heart and the term is often used interchangeably with "cardiovascular disease," which generally refers to conditions that involve narrowed or blocked blood vessels that can lead to a heart attack. Heart disease is also a leading cause of death globally, one in five deaths in the U.S. is caused by heart disease and heart disease cost the U.S. 239.9 billion dollars each year. There are many risk factors that directly associates with the presence of heart diease in patients, such as age and sex. With that said, the goal for us today is to classify patients with or without heart disease by distinguishing the likelihood using the risk factors. The dataset we will be using to perform this process on will be the 1988 Hungary Heart Disease database. 

In [36]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [71]:
# cleaveland_data <- read_tsv("data/cleve.mod", skip = 20)

url <- "https://raw.githubusercontent.com/ANGUO17/dsci-100-2023w2-group-06/main/heart%2Bdisease/reprocessed.hungarian.data"

data <- read.table(url)   

#Age; sex; chest pain type (angina, abnang, notang, asympt)
#%  Trestbps (resting blood pres); cholesteral; fasting blood sugar < 120
#%  (true or false); resting ecg (norm, abn, hyper); max heart rate; 
#%  exercise induced angina (true or false); oldpeak; slope (up, flat, down)
#%  number of vessels colored (???); thal (norm, fixed, rever). Finally, the
#%  class is either healthy (buff) or with heart-disease (sick).
    
head(data)




ERROR: Error in parse(text = x, srcfile = src): <text>:5:27: unexpected symbol
4: 
5: data <- read.table(url)   col_names
                             ^
